In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

In [ ]:
def gompertz(t,m,A,l):
    return A * np.exp(-np.exp(m*np.e/A*(l-t) + 1))

In [ ]:
# kbase = GPy.kern.RBF(1,name='base')
# kbio = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1)])
# ktech = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1), GPy.kern.RBF(1,name='tech', variance=.05)])

In [ ]:
nbio = 3
ntech = 3
ntot = nbio * ntech

x = np.zeros((50*ntot,3))
x[:,0] = np.tile(np.linspace(-1,1), ntot)
x[:,1] = np.repeat(np.arange(nbio), ntech*50)
x[:,2] = np.repeat(np.arange(nbio*ntech), 50)

In [ ]:
baseVariance, bioVariance, techVariace = 1, .02, .001

kbase = GPy.kern.RBF(1,name='base',variance=baseVariance)
kbio = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='bio', variance=bioVariance), index_dim=-2);
ktech = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='tech', variance=techVariace))

In [ ]:
plt.imshow(kbio.K(x))

plt.figure()
plt.imshow(ktech.K(x))

In [ ]:
sigma = .001
# noise = np.eye(50*ntot)*sigma

f = gompertz(x[:,0]+1, 2, 1, .4)

np.random.seed(1)

plt.figure(figsize=(12,4))
for i,kern in enumerate([None, kbio, ktech]):

    mu = np.zeros(x.shape[0])
    
    if not kern is None:
        cov = kern.K(x)
        f += scipy.stats.multivariate_normal.rvs(mu,cov)
        
    extraVariance = sum([0, bioVariance, techVariace][i+1:])
    noise = np.eye(50*ntot)*(sigma+extraVariance)
    
    s = scipy.stats.multivariate_normal.rvs(f,noise).reshape((ntot,50)).T
    
    plt.subplot(1,3,i+1)
    
#     if i > 0:
#         for j,z in enumerate(x[::50,i]):
#             k = np.unique(x[:,i]).tolist().index(z)
#             plt.plot(s[:,j],color='C%d'%k);
#     else:
#         plt.plot(s,c='k');
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,j],color='C%d'%k);
            
    plt.ylim(-.25,1.3)
        
plt.savefig("figures/simulated-data.pdf",bbox_inches='tight')